# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# TO-DO: Create a Keyspace 
try:
    print("Creating keyspace: sparkifykeyspace")
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkifykeyspace
        WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};
    """)
    print("Creating keyspace: DONE.")

except Exception as e:
    print(e)

Creating keyspace: sparkifykeyspace
Creating keyspace: DONE.


#### Set Keyspace

In [8]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkifykeyspace')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4


try:
    print("Creating table: songs_by_session_and_item")
    session.execute("""
        CREATE TABLE IF NOT EXISTS sparkifykeyspace.songs_by_session_and_item (
           sessionId int,
           itemInSession int,
           artist text,
           song text,
           length float,
           PRIMARY KEY (sessionId, itemInSession)
        ) WITH COMMENT = 'Q1: artist, song title and songlength that was during sessionId 338 and itemInSession 4';
    """)
    print("Creating table: Done")
except Exception as e:
    print(e)

Creating table: songs_by_session_and_item
Creating table: Done


In [10]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into sparkifykeyspace.songs_by_session_and_item (sessionId, itemInSession, artist, song, length)"
        query = query + "values (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`

        # header sequence:
        # artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
        insert_data = (int(line[8]), int(line[3]), line[0], line[9], float(line[5]))
#         print(insert_data)

        session.execute(query, insert_data)
#         print("inserted.\n")


print("inserted.\n")

inserted.



#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
r = session.execute('SELECT * FROM sparkifykeyspace.songs_by_session_and_item WHERE sessionId = 338 AND itemInSession = 4;')
print(r.current_rows)

[Row(sessionid=338, iteminsession=4, artist='Faithless', length=495.30731201171875, song='Music Matters (Mark Knight Dub)')]


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [14]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

try:
    print("Creating table: events_by_user_by_session")
    session.execute("""
        CREATE TABLE IF NOT EXISTS sparkifykeyspace.events_by_user_by_session (
            userId int,
            sessionId int,
            itemInSession int,
            artist text,
            song text,
            firstName text,
            lastName text,
            PRIMARY KEY ((userId, sessionId), itemInSession)
        ) WITH comment = 'q2: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182'
        AND CLUSTERING ORDER BY (itemInSession ASC);
    """)
    print("Creating table: DONE")
except Exception as e:
    print(e)
                    

Creating table: events_by_user_by_session
Creating table: DONE


In [15]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into sparkifykeyspace.events_by_user_by_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "values (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`

        # header sequence:
        # 0_artist,1_firstName,2_gender,3_itemInSession,4_lastName,5_length,6_level,7_location,8_sessionId,9_song,10_userId
        insert_data = (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4])
#         print(insert_data)

        # session.execute(query, insert_data)

        session.execute(query, insert_data)
#         print("inserted.\n")

print("inserted.\n")


inserted.



In [16]:
r = session.execute("""
    SELECT artist, song, firstName, lastName
    FROM sparkifykeyspace.events_by_user_by_session
    WHERE userId = 10 AND sessionId = 182;
""")

print(r.current_rows)

[Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz'), Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz'), Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz'), Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')]


In [18]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


try:
    print("Creating table: users_by_song")
    session.execute("""
        CREATE TABLE IF NOT EXISTS sparkifykeyspace.users_by_song (
            song text,
            firstName text,
            lastName text,
            PRIMARY KEY (song, firstName, lastName)
        ) WITH comment = 'q3: every user name (first and last) in my music app history who listened to the song `All Hands Against His Own`';
    """)
    print("Creating table: DONE")
except Exception as e:
    print(e)                 

Creating table: users_by_song
Creating table: DONE


In [19]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into sparkifykeyspace.users_by_song (song, firstName, lastName)"
        query = query + "values (%s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`

        # header sequence:
        # 0_artist,1_firstName,2_gender,3_itemInSession,4_lastName,5_length,6_level,7_location,8_sessionId,9_song,10_userId
        insert_data = (line[9], line[1], line[4])
#         print(insert_data)

        # session.execute(query, insert_data)

        session.execute(query, insert_data)
#         print("inserted.\n")

print("inserted.\n")


inserted.



In [20]:
r = session.execute("""
    SELECT firstName, lastName
    FROM sparkifykeyspace.users_by_song
    WHERE song = 'All Hands Against His Own';
""")

print(r.current_rows)

[Row(firstname='Jacqueline', lastname='Lynch'), Row(firstname='Sara', lastname='Johnson'), Row(firstname='Tegan', lastname='Levine')]


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [21]:
try:
    session.execute("DROP TABLE IF EXISTS sparkifykeyspace.songs_by_session_and_item;")
except Exception as e:
    print(e)

try:
    session.execute("DROP TABLE IF EXISTS sparkifykeyspace.events_by_user_by_session;")
except Exception as e:
    print(e)

try:
    session.execute("DROP TABLE IF EXISTS sparkifykeyspace.users_by_song;")
except Exception as e:
    print(e)

try:
    session.execute("DROP KEYSPACE IF EXISTS sparkifykeyspace;")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()